# 15 - Lidando com dimensões

In [8]:
import thinkbayes

## 15.1 - Bactérias do umbigo

- Projeto com objetivo de identificar as especies de bactérias que pode ser encontrado em umbigo humanos;
- É de interesse do microbioma humano, o conjunto de migroorganismo que vive na pele humana e partes do corpo;

- Os pesquisadores coletaram amostras de 60 volumtários;
- Usou o método multiplex pyrosequencing para extrair e sequenciar fragmentos de rDNA 16S;

Podemos usar esses dados para responder a várias questões relacionadas:

- Com base no número de espécies observadas, podemos estimar o número total de espécies no ambiente?
- Podemos estimar a prevalência de cada espécie, ou seja, a fração da população total pertencente a cada espécie?
- Se estamos planejando coletar amostras adicionais, podemos prever quantas novas espécies poderemos descobrir?
- Quantas leituras adicionais são necessárias para aumentar a fração de espécies observadas até um determinado limite?

**Problema das Espécies Invisíveis**

## 15.2 - Leões, tigres e ursos

Problema simples onde conhecemos 3 espécies;
- 3 Leões, 2 Tigres e 1 Urso;

Chance igual de observar qualquer animal, o de vermos cada espécie é dada pela distribuição multinomial;
A verossimilhança de ver 3 leões, 2 tigres e um urso é proporcional a
```python
    p_lion**3 * p_tiger**2 * p_bear**1
```

- **Distribuição Beta**: Uma abordagem atentadora, mas não correta;

```python
    beta = thinkbayes.Beta()
    beta.Update((3, 3))
    print beta.MaximumLikelihood()
```
- `p_lion = 50%`
- `p_tiger = 33%`
- `p_bear = 17%`

**Problemas**
1. A a priori não é adequada para esse tipo de problema:
    - Distribuição uniforme para cada espécie;
2. As distribuições para cada espécie não pode ser independente:
    - Precisamos de uma distribuição conjunta;
    
**Solução**
- Usar uma distribuição Dirichlet, assim, resolvendo os dois problemas;
    - Descreve a distribuição conjunta de `p_lion`, `p_tiger` e `p_bear`.
    - É a generalização multidimensional da distribuição beta;
    - É descrita por n parâmetros, escritos de `α1` a `αn`.
    - A distribuição marginal para cada prevalência é uma distribuição beta;

In [6]:
import numpy

class Dirichlet(object):

    def __init__(self, n):
        self.n = n
        self.params = numpy.ones(n, dtype=numpy.int)
        
    def MarginalBeta(self, i):
        alpha0 = self.params.sum()
        alpha = self.params[i]
        return thinkbayes.Beta(alpha, alpha0-alpha)
    
    def Update(self, data):
        m = len(data)
        self.params[:m] += data

In [3]:
dirichlet = thinkbayes.Dirichlet(3)
for i in range(3):
    beta = dirichlet.MarginalBeta(i)
    print(beta.Mean())

0.3333333333333333
0.3333333333333333
0.3333333333333333


```python
data = [3, 2, 1]
dirichlet = Dirichlet(3)
dirichlet.Update(data)

for i in range(3):
    beta = dirichlet.MarginalBeta(i)
    pmf = beta.MakePmf()
    print(i, pmf.Mean())

0 0.44 
1 0.33
2 0.22    
```


![](post_15_2.png)

Distribuição das prevalências para três espécies

## 15.3 - A versão hierárquica

**Problema simples:** Se soubermos quantas espécies existem, podemos estimar a prevalência de cada uma.

**Problema original:** Estimando o número total de espécies
- Definiu **Meta-Suite**, que é uma Suite que contém outras Suites como hipóteses.
- Nível superior contém hipóteses sobre o número de espécies; 
- Nível inferior contém hipóteses sobre prevalências.

In [19]:
class Species(thinkbayes.Suite):

    def __init__(self, ns):
        hypos = [thinkbayes.Dirichlet(n) for n in ns]
        thinkbayes.Suite.__init__(self, hypos)
        
    def Update(self, data):
        thinkbayes.Suite.Update(self, data)
        for hypo in self.Values():
            hypo.Update(data)
            
    def Likelihood(self, data, hypo):
        dirichlet = hypo
        like = 0
        for i in range(1000):
            like += dirichlet.Likelihood(data)

        return like

In [25]:
ns = range(3, 30)
suite = Species(ns)

- Assumimos que qualquer valor nesse intervalo é igualmente provável.
- O comprimento de `data` é o número de espécies observadas. 
- Se vemos mais espécies do que pensávamos existir, a probabilidade é 0.

## 15.4 - Amostragem aleatória

Existem duas maneiras de gerar uma amostra aleatória a partir de uma distribuição Dirichlet:
1. Usar as distribuições beta marginais;
2. Selecionar valores de n distribuições gama e normalizar dividindo pelo total

```python
# class Dirichlet

    def Random(self):
        p = numpy.random.gamma(self.params)
        return p / p.sum()
```

Distribuição a posteriori de n;

```python
    def DistOfN(self):
        pmf = thinkbayes.Pmf()
        for hypo, prob in self.Items():
            pmf.Set(hypo.n, prob)
        return pmf
```

Percorre as hipóteses de nível superior e acumula a probabilidade de cada n.

![](post_15_4.png)

Distribuição a posteriori de n

- O valor mais provável é 4;
- Valores de 3 a 7 são razoavelmente prováveis;
- A probabilidade de haver 29 espécies é baixa o suficiente para ser insignificante;

- A priori -> uniforme;
- Com mais informações sobre as espécies, poderemos escolher uma a priori diferente;

## 15.5 - Otimização

- Poucas linhas de código para a solução;
- É lento;
    - Bom para poucas espécies;
    - Não é bom para o problema do umbigo;

As próximas seções apresentam uma série de otimizações:
- Não precisamos realmente de n objetos Dirichlet; podemos armazenar os parâmetros no nível superior da hierarquia.
- Mesmo conjunto de valores aleatórios para todas as hipóteses.
- Fazer as atualizações de uma espécie por vez;
- combina as sub-hipóteses no nível superior e usa operações numpy de array para acelerar as coisas.

## 15.6 - Colapsando a hierarquia

In [28]:
class Species2(object):
    
    def __init__(self, ns):
        self.ns = ns
        self.probs = numpy.ones(len(ns), dtype=numpy.double)
        self.params = numpy.ones(self.high, dtype=numpy.int)
    
    def Update(self, data):
        like = numpy.zeros(len(self.ns), dtype=numpy.double)
        for i in range(1000):
            like += self.SampleLikelihood(data)

        self.probs *= like
        self.probs /= self.probs.sum()

        m = len(data)
        self.params[:m] += data
    
    def SampleLikelihood(self, data):
        gammas = numpy.random.gamma(self.params)

        m = len(data)
        row = gammas[:m]
        col = numpy.cumsum(gammas)

        log_likes = []
        for n in self.ns:
            ps = row / col[n-1]
            terms = data * numpy.log(ps)
            log_like = terms.sum()
            log_likes.append(log_like)

        log_likes -= numpy.max(log_likes)
        likes = numpy.exp(log_likes)

        coefs = [thinkbayes.BinomialCoef(n, m) for n in self.ns]
        likes *= coefs

        return likes

```python
__init__()
```
- `ns` é a lista de valores hipotéticos para n; 
- `probs` é a lista de probabilidades correspondentes;
- `params` é a sequência dos parâmetros de Dirichlet;

--------------------------

```python
Update()
```
Atualiza os dois níveis da hierarquia: 
1. A probabilidade de cada valor de n;
2. Os parâmetros Dirichlet:

-------------------------------

```python
SampleLikelihood()
```
- Duas oportunidades de otimização:
    - O número hipotético(n) de espécies excede o número observado(m), precisamos apenas dos primeiros m.
    - Número de espécies for grande, a probabilidade dos dados pode ser muito pequena, usamos log.

- A versão otimizada é menos legível e mais suscetível a erros do que a original.
- Usar a versão simples para fazer um teste de regressão;

## 15.7 - Mais um problema

À medida que o número de espécies observadas aumenta, leva mais iterações para convergir para uma boa resposta.

As prevalências que escolhemos da distribuição de Dirichlet;

A maioria das iterações não fornece nenhuma contribuição útil para a probabilidade total;

Solução:
- A atualizar da distribuição a priori com todo o conjunto de dados ou dividi-la em uma série de atualizações;

Essa versão representa as hipóteses como uma lista de objetos Dirichlet;


```python
# class Species4

    def Likelihood(self, data, hypo):
        dirichlet = hypo
        like = 0
        for i in range(self.iterations):
            like += dirichlet.Likelihood(data)

        # correct for the number of unseen species the new one
        # could have been
        m = len(data)
        num_unseen = dirichlet.n - m + 1
        like *= num_unseen

        return like
```

fator `num_unseen`?
    

## 15.8 - Ainda não terminamos